In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Análisis Preliminar de Datos

Antes de levantar los archivos, cambiamos su atributo `encoding` a `utf-8`. <br>
Luego elegimos el encoding (puede ser ANSI o Latin-1, por el momento, debido a las tildes).

In [2]:
#Levantamos los datasets en crudo (archivos .csv)
df_clientes     =pd.read_csv('./Datasets/Clientes.csv'   , encoding='latin-1', sep=';')
df_compras      =pd.read_csv('./Datasets/Compra.csv'     , encoding='latin-1', sep=',')
df_gastos       =pd.read_csv('./Datasets/Gasto.csv'      , encoding='latin-1', sep=',')
df_localidades  =pd.read_csv('./Datasets/Localidades.csv', encoding='latin-1', sep=',')
df_proveedores  =pd.read_csv('./Datasets/Proveedores.csv', encoding='latin-1', sep=',')
df_sucursales   =pd.read_csv('./Datasets/Sucursales.csv' , encoding='latin-1', sep=';')
df_ventas       =pd.read_csv('./Datasets/Venta.csv'      , encoding='latin-1', sep=',')

Las columnas están nombradas sin seguir ninguna regla. Algunas en mayúsculas, otras en minúsculas, con `camelcase` u otro tipo de separación entre palabras, cómo la separación mediante `underscore`.
Además,ciertos nombres no son lo suficientemente descriptivos para brindarnos la información necesaria sobre lo que contiene la columna.<br>
Este mismo invonveniente tenemos en el contenido de ciertas columnas. Los datos no siguen un estándar de escritura.

Mencionamos algunas incongruencias en el dataset **Clientes.csv**
- Que es X o Y? No sabemos a que hace referencia.
- ID en mayúsculas y el resto de columnas capitalizadas.
- El domicilio se carga en mayúsculas?.

In [3]:
#Vemos las incongruencias en un solo registro...
df_clientes.head(1)

,ID,Provincia,Nombre_y_Apellido,Domicilio,Telefono,Edad,Localidad,X,Y,col10
0,1,Buenos Aires,HEBER JONI SANTANA,LAS HERAS Y BAT. 24 DE FEBRERO 4150 RINCON DE...,42-5161,58,LOMA VERDE,"-58,81850307","-34,30997088",NaN


No existe una `normalización` con respecto a los datos insertados. Es decir, nos encontramos con diversas maneras de referir al mismo dato. <br>
Tomamos el ejemplo concreto de la tabla **Sucursales.csv**. Al ingresar el nombre de una Provincia, se debe determinar la manera correcta de ingestarlo, y ese marco debe ser respetado en las futuras ingestas. <br>
¿De cuantas maneras distintas se puede escribir un mismo dato? No es algo que busquemos saber.
En su lugar, determinemos, por ejemplo: a partir de ahora Buenos Aires se debe incertar de la 
siguiente manera -> `'Ciudad de Buenos Aires'`. Siempre así, no se aceptan variaciones.
Para evitar tener que lidiar con 'Bs. As', 'Buenos Aires', 'C. de Buenos Aires', etc etc

In [4]:
#Observemos que en solo 5 registros observamos que no existe normalización en Localidad y Provincia.
df_sucursales.head()

,ID,Sucursal,Direccion,Localidad,Provincia,Latitud,Longitud
0,1,Cabildo,Av. Cabildo 1342,Ciudad de Buenos Aires,Ciudad de Buenos Aires,"-34,5678060","-58,4495720"
1,2,Palermo 1,Guatemala 5701,CABA,CABA,"-34,5790350","-58,4335660"
2,3,Palermo 2,Gral. Lucio Norberto Mansilla 2668,CABA,C deBuenos Aires,"-34,5959660","-58,4051500"
3,4,Corrientes,Av. Corrientes 2352,Ciudad de Buenos Aires,Bs As,"-34,6046850","-58,3987640"
4,5,Almagro,Venezuela 3650,Capital,Bs.As.,"-34,6173080","-58,4161790"


## Evaluación de Calidad de Datos

El dataset **Clientes.csv** posee una columna completa con valores nulos.Procedemos a eliminarla debido a que no aporta ningún tipo de valor y entorpece el análisis.

In [90]:
df_clientes.drop(columns='col10', inplace=True)

Obtenemos un análisis de los registros y datos de todos los datasets.

In [154]:
dataFrames = [df_clientes, df_compras, df_gastos, df_localidades, df_proveedores, df_sucursales, df_ventas]
names = ['df_clientes', 'df_compras', 'df_gastos', 'df_localidades', 'df_proveedores', 'df_sucursales', 'df_ventas']

In [155]:
total_datos = []
sin_null = []
con_null = []
total_registros = []
total_registros_null = []

for i, df in enumerate(dataFrames):
    #Datos null inclusive
    total_datos.append( df.shape[0] * df.shape[1] )
    #Datos sin null
    sin_null.append(df.count().sum())
    #Cantidad de Registros
    total_registros.append( len(df) )
    #Datos null
    null_columnas = df.isna().sum()
    con_null.append( null_columnas.sum() )
    #Obtenemos la cantidad de registros que poseen al menos un dato nulo
    is_null = df.isnull()
    row_has_null = is_null.any(axis=1)
    total_registros_null.append( len(df[row_has_null]) )

In [156]:
data = pd.DataFrame({
'Dataset': names,
'Datos Total': total_datos, 
'Datos sin null': sin_null, 
'Datos Nulos': con_null, 
'Cantidad Registros': total_registros, 
'Cant de Registros con Null': total_registros_null
})

In [161]:
data

,Dataset,Datos Total,Datos sin null,Datos Nulos,Cantidad Registros,Cant de Registros con Null
0,df_clientes,24048,23787,261,2672,145
1,df_compras,103851,103484,367,11539,367
2,df_gastos,43200,43200,0,8640,0
3,df_localidades,57988,56949,1039,4142,543
4,df_proveedores,98,96,2,14,2
5,df_sucursales,217,217,0,31,0
6,df_ventas,466450,464634,1816,46645,1800


Guardamos el resultado en un archivo para su futura consulta.

In [163]:
data.to_csv('analisis.csv', index=False)